Now import all necessary libraries

In [7]:
import math

import cv2
import numpy as np
import glob

Variable debug to see intermediate results

In [8]:
debug = False

Now we import camera matrix and distortion coefficient

In [9]:
from matplotlib.pyplot import draw

K = np.load ( "framesVideoCalibration/uncalibrated/K.npy" )
dist = np.load ( "framesVideoCalibration/uncalibrated/dist.npy" )

listImages = glob.glob ( "framesVideo1/uncalibrated/*.jpg" )

# for each image in the list
for filename in listImages :

	# read the actual image
	img = cv2.imread ( filename , 0 )

	cv2.imwrite ( filename.replace ( "uncalibrated" , "backgroundremoved" ) , img )

Now we start to elaborate the video

In [10]:
# get list of frames of video 1
"""listImages = glob.glob ( "framesVideo1/uncalibrated/*.jpg" )

# for each image in the list
for filename in listImages :

	# read the actual image
	img = cv2.imread ( filename , 0 )

	cv2.imwrite ( filename.replace ( "uncalibrated" , "backgroundremoved" ) , equ )"""



'listImages = glob.glob ( "framesVideo1/uncalibrated/*.jpg" )\n\n# for each image in the list\nfor filename in listImages :\n\n\t# read the actual image\n\timg = cv2.imread ( filename , 0 )\n\n\tcv2.imwrite ( filename.replace ( "uncalibrated" , "backgroundremoved" ) , equ )'

In [11]:
import numpy as np
import cv2
import glob

# get list of frames of video 1
listImages = glob.glob ( "framesVideo1/uncalibrated/*.jpg" )

# for each image
for image in listImages :

	# open the actual image in color mode
	actual_image = cv2.imread ( image , cv2.IMREAD_COLOR )

	# open the actual image in gray mode
	gray_actual_image = cv2.imread ( image , cv2.IMREAD_GRAYSCALE )

	# make binary threshold on gray actual image
	_ , threshold_gray_actual_image = cv2.threshold ( gray_actual_image , 190 , 255 , cv2.THRESH_BINARY )

	# detect shape in threshold gray actual image using regions with same colors and intensity
	contours , _ = cv2.findContours ( threshold_gray_actual_image , cv2.RETR_TREE , cv2.CHAIN_APPROX_SIMPLE )

	# for each contour
	for contour in contours :

		# calculate the area of the actual region
		area_actual_region = cv2.contourArea ( contour )

		# if the area is acceptable
		if area_actual_region > 200 :

			# approximate the shape of the actual polygon
			approx = cv2.approxPolyDP ( contour , 0.01 * cv2.arcLength ( contour , True ) , True )

			# if we have 5 edges in the actual region
			if len ( approx ) == 5 :

				# we can dra contours of the actual region but in the colored image
				cv2.drawContours ( actual_image , [ approx ] , 0 , (0 , 0 , 255) , 2 )

	# save the image in the correct folder
	cv2.imwrite ( image.replace ( "uncalibrated" , "polygonborders" ) , actual_image )
	cv2.imwrite ( image.replace ( "uncalibrated" , "binarythreshold" ) , threshold_gray_actual_image )

In [24]:
import numpy as np
import cv2
import glob
import math

# get list of frames of video 1
listImages = glob.glob ( "framesVideo1/uncalibrated/*.jpg" )

# for each image
for image in listImages :

	# open the actual image in color mode
	actual_image = cv2.imread ( image , cv2.IMREAD_COLOR )

	# open the actual image in gray mode
	gray_actual_image = cv2.imread ( image , cv2.IMREAD_GRAYSCALE )

	# make binary threshold on gray actual image
	_ , threshold_gray_actual_image = cv2.threshold ( gray_actual_image , 190 , 255 , cv2.THRESH_BINARY )

	# save the threshold gray actual image
	cv2.imwrite ( image.replace ( "uncalibrated" , "binarythreshold" ) , threshold_gray_actual_image )

	# detect shape in threshold gray actual image using regions with same colors and intensity
	contours , hierarchy = cv2.findContours ( image = threshold_gray_actual_image , mode = cv2.RETR_TREE , method = cv2.CHAIN_APPROX_NONE )
	#print ( contours [ 164 ] )
	#cv2.drawContours(image=actual_image, contours=contours, contourIdx=-1, color=(0, 255, 0), thickness=2, lineType=cv2.LINE_AA)

	#save the index of actual contour
	index_actual_contour = 0

	# for each contour in threshold_gray_actual_image
	for contour in contours :

		# calculate the area of the actual region
		area_actual_region = cv2.contourArea ( contour )

		# approximate the shape of the actual polygon
		approx = cv2.approxPolyDP ( contour , 0.01 * cv2.arcLength ( contour , True ) , True )

		# if the area of the actual region is bigger than a minimum value
		if area_actual_region > 100 :

			# if we have 5 edges in the actual region
			if len ( approx ) == 5 :

				# calculate the perimeter of the contour
				perimeter = cv2.arcLength ( approx , True )

				# calculate the average of one edge
				avg_perimeter = perimeter / len ( approx )
				print ( avg_perimeter )

				# save index of concave vertex
				first_edge_concave_vertex_clockwise = 0

				# for each edge
				for i in range ( 0 , len ( approx ) ) :

					# calculate the distance between the previous and the next one
					previous = math.dist ( [ approx [ i ] [ 0 ] [ 0 ] , approx [ i ] [ 0 ] [ 1 ] ] , [ approx [ (i - 1) % len ( approx ) ] [ 0 ] [ 0 ] , approx [ (i - 1) % len ( approx ) ] [ 0 ] [ 1 ] ] )
					next = math.dist ( [ approx [ i ] [ 0 ] [ 0 ] , approx [ i ] [ 0 ] [ 1 ] ] , [ approx [ (i + 1) % len ( approx ) ] [ 0 ] [ 0 ] , approx [ (i + 1) % len ( approx ) ] [ 0 ] [ 1 ] ] )

					# if they are both less than average edge
					if previous < avg_perimeter and next < avg_perimeter :

						# this is the concave vertex
						first_edge_concave_vertex_clockwise = i

						# now draw circles of different color from the concave vertex in clockwise
						for j in range ( 0 , len ( approx ) ) :

							# if it is the concave vertex
							if j == 0 :

								# red circle
								actual_image = cv2.circle ( actual_image , (int ( approx [ (first_edge_concave_vertex_clockwise + j) % len ( approx ) ] [ 0 ] [ 0 ] ) , int ( approx [ (first_edge_concave_vertex_clockwise + j) % len ( approx ) ] [ 0 ] [ 1 ] )) , radius = 4 , color = (0 , 0 , 255) , thickness = -1 )

							# one vertex after the concave vertex
							elif j == 1 :

								# green circle
								actual_image = cv2.circle ( actual_image , (int ( approx [ (first_edge_concave_vertex_clockwise + j) % len ( approx ) ] [ 0 ] [ 0 ] ) , int ( approx [ (first_edge_concave_vertex_clockwise + j) % len ( approx ) ] [ 0 ] [ 1 ] )) , radius = 4 , color = (0 , 255 , 0) , thickness = -1 )

							# two vertex after the concave vertex
							elif j == 2 :

								# blue circle
								actual_image = cv2.circle ( actual_image , (int ( approx [ (first_edge_concave_vertex_clockwise + j) % len ( approx ) ] [ 0 ] [ 0 ] ) , int ( approx [ (first_edge_concave_vertex_clockwise + j) % len ( approx ) ] [ 0 ] [ 1 ] )) , radius = 4 , color = (255 , 0 , 0) , thickness = -1 )

							# three vertex after the concave vertex
							elif j == 3 :

								# yellow circle
								actual_image = cv2.circle ( actual_image , (int ( approx [ (first_edge_concave_vertex_clockwise + j) % len ( approx ) ] [ 0 ] [ 0 ] ) , int ( approx [ (first_edge_concave_vertex_clockwise + j) % len ( approx ) ] [ 0 ] [ 1 ] )) , radius = 4 , color = (0 , 255 , 255) , thickness = -1 )

							# four vertex after the concave vertex
							elif j == 4 :

								# pink circle
								actual_image = cv2.circle ( actual_image , (int ( approx [ (first_edge_concave_vertex_clockwise + j) % len ( approx ) ] [ 0 ] [ 0 ] ) , int ( approx [ (first_edge_concave_vertex_clockwise + j) % len ( approx ) ] [ 0 ] [ 1 ] )) , radius = 4 , color = (255 , 0 , 255) , thickness = -1 )

						# we can exit from the for of approx edges
						break

				#actual_image = cv2.putText ( actual_image , str ( i ) , (approx [ i ] [ 0 ] [ 0 ] , approx [ i ] [ 0 ] [ 1 ]) , cv2.FONT_HERSHEY_SIMPLEX , 1 , (0 , 0 , 255) , 2 )

				#print ( cv2.isContourConvex ( np.array ( [ np.array ( approx [ i ] ) , np.array ( approx [ (i + 1) % 5 ] ) , np.array ( approx [ (i + 2) % 5 ] ) ] ) ) )

				#approx [ 0 ] [ (i + 1) % len ( approx ) ]

				#print ( [ approx [ i ] [ 0 ] [ 0 ] , approx [ i ] [ 0 ] [ 1 ] ] , [ approx [ (i + 1) % len ( approx ) ] [ 0 ] [ 0 ] , approx [ (i + 1) % len ( approx ) ] [ 0 ] [ 1 ] ] )

				#a = math.dist ( [ approx [ i ] [ 0 ] [ 0 ] , approx [ i ] [ 0 ] [ 1 ] ] , [ approx [ (i + 1) % len ( approx ) ] [ 0 ] [ 0 ] , approx [ (i + 1) % len ( approx ) ] [ 0 ] [ 1 ] ] )
				#print ( np.array ( [ np.array ( approx [ i ] ) , np.array ( approx [ (i + 1) % 5 ] ) , np.array ( approx [ (i + 2) % 5 ] ) ] ) )
				#print ( "\n" )
				#print ( "-------------------------------------" )
				#print ( perimeter )
				#print ( "\n\n\n" )

				"""if cv2.isContourConvex(edge):
                    cv2.drawContours ( actual_image , [ edge ] , 0 , (255 , 0 , 0) , 5 )

                else:
                    cv2.drawContours ( actual_image , [ edge ] , 0 , (0 , 255 , 255) , 5 )"""

				#print ( hierarchy [ 0 ] [ i ] )

				# initialize the list of children of actual polygon as empty
				children_contour = [ ]

				# we know that contours have form [next, previous, first child, parent]
				children = hierarchy [ 0 ] [ index_actual_contour ] [ 2 ]

				# while there are other childrens
				while children != -1 :

					# append actual children to the previous list
					children_contour.append ( children )

					# obtain the right contour
					children = hierarchy [ 0 ] [ children ] [ 0 ]

				# if there are children in actual contour
				if len ( children_contour ) > 0 :

					# we can draw contours of the actual region but in the colored image
					cv2.drawContours ( actual_image , [ contour ] , 0 , (0 , 0 , 255) , 1 )

				# for each children
				for single_children in children_contour :

					# draw contour
					cv2.drawContours ( actual_image , contours , single_children , (0 , 255 , 0) , 1 )



					M = cv2.moments(contour)
					cX = int(M["m10"] / M["m00"])
					cY = int(M["m01"] / M["m00"])

					# draw the center of the shape on the image
					cv2.circle(actual_image, (cX, cY), 1, (0, 0, 255), -1)

		# increment the index of actual contour
		index_actual_contour = index_actual_contour + 1

	"""i = 0
	# for each contour in threshold_gray_actual_image
	for contour in contours_threshold_gray_actual_image :

		# calculate the area of the actual region
		area_actual_region = cv2.contourArea ( contour )

		# approximate the shape of the actual polygon
		approx = cv2.approxPolyDP ( contour , 0.01 * cv2.arcLength ( contour , True ) , True )

		# if the area is acceptable
		if 200 <= area_actual_region <= 400 :

			if 6 <= len ( approx ) <= 20 :

				for j in list_black_polygon :

					if hierarchies [ j , i , 3 ] == -1 :

						# we can dra contours of the actual region but in the colored image
						cv2.drawContours ( actual_image , [ approx ] , 0 , (0 , 255 , 0) , 2 )

						print ( "cciao" )

		i = i + 1"""
	cv2.imwrite ( image.replace ( "uncalibrated" , "polygonborders" ) , actual_image )




81.4210147857666
72.21554851531982
90.33742332458496
89.47839584350587
82.332963180542
74.09707641601562
71.3818344116211
78.963822555542
86.52598571777344
68.15137157440185
90.64468803405762
77.12793083190918
88.3177303314209
85.4444622039795
68.55131187438965
85.03842086791992
77.91093444824219
90.23221817016602
69.27773990631104
89.93633346557617
86.30237998962403
77.24432754516602
71.17091026306153
74.85419082641602
52.34407424926758
83.98882904052735
62.05558109283447
89.48413619995117
71.23492469787598
90.91742477416992
86.1545482635498
60.19568099975586
84.91469230651856
77.63333473205566
91.0856746673584
86.8559455871582
79.05086135864258
71.48715019226074
74.71505088806153
82.97615394592285
52.976641464233396
62.39275569915772
89.1024772644043
71.91325092315674
89.54193496704102
80.374560546875
88.07823104858399
59.97734260559082
84.40960159301758
77.19036560058593
89.85238418579101
91.30247039794922
87.37807693481446
78.58099403381348
71.78012008666992
74.53163719177246
82.81